# Import Libraries

In [ ]:
from PyPDF2 import PdfReader
import google.generativeai as genai
import os
import json
from dotenv import load_dotenv

In [4]:
!pip install PyPDF2

Defaulting to user installation because normal site-packages is not writeable
  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [7]:
!pip install google.generativeai

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 160.8/160.8 kB 9.4 MB/s eta 0:00:00
     -------------------------------------- 431.5/431.5 kB 4.5 MB/s eta 0:00:00
     -------------------------------------- 760.0/760.0 kB 6.8 MB/s eta 0:00:00
     ---------------------------------------- 12.3/12.3 MB 3.8 MB/s eta 0:00:00
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl (9.3 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)
     ---------------------------------------- 4.3/4.3 MB 4.2 MB/s eta 0:00:00


In [11]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


# Load env variables

In [8]:
load_dotenv()

True

# API KEY Set up

In [9]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Model define

In [10]:
model = genai.GenerativeModel("models/gemini-1.5-pro")

# Analyze resume using llm

In [11]:
def resumes_details(resume):
    prompt = f"""
    You are a resume parsing assistant. Extract the following details from the given resume text. Ensure to return "N/A" if information is missing or unclear.
    
    Resume text:
    {resume}
    
    Extract and include the following in a structured JSON format:
    1. Full Name
    2. Contact Number
    3. Email Address
    4. Location
    5. Skills (separate Technical and Non-Technical Skills)
    6. Education (degree, institution, graduation year)
    7. Work Experience (company name, role, responsibilities, years of experience)
    8. Certifications
    9. Languages spoken
    
    Ensure the output is structured like this:

    {{
        "Full Name": "...",
        "Contact Number": "...",
        "Email Address": "...",
        "Location": "...",
        "Skills": {{
            "Technical Skills": [...],
            "Non-Technical Skills": [...]
        }},
        "Education": [
            {{
                "Degree": "...",
                "Institution": "...",
                "Years": "..."
            }},
            ...
        ],
        "Work Experience": [
            {{
                "Job Title": "...",
                "Company Name": "...",
                "Years of Experience": "...",
                "Responsibilities": ["..."]
            }},
            ...
        ],
        "Certifications": [...],
        "Languages spoken": [...]
    }}
    """

    response = model.generate_content(prompt).text
    return response

# Upload File and extract text

In [12]:
def upload_resume(file_path):
    """
    Function to upload and read a resume PDF file.
    """
    if not file_path.endswith('.pdf'):
        return {"error": "The uploaded file is not a PDF."}

    text = ""
    reader = PdfReader(file_path)
    for page in reader.pages:
        text += page.extract_text()

    return text

In [13]:
resume_file_path = "D:\ml\project\Resume Parser/Preeti.pdf"  
resume_text = upload_resume(resume_file_path)

# Defining some parameters in regex

In [14]:
if isinstance(resume_text, dict) and "error" in resume_text:
    print(resume_text["error"])
else:
    # Parse the resume text using the generative model
    response = resumes_details(resume_text)

    # Clean the response
    response_clean = response.replace("```json", "").replace("```", "").strip()

# Load the cleaned response into a dictionary
try:
    data = json.loads(response_clean)
except json.JSONDecodeError:
    print("Failed to parse resume details")
    data = {}

# Set default values for missing fields
full_name = data.get("Full Name", "N/A")
contact_number = data.get("Contact Number", "N/A")
email_address = data.get("Email Address", "N/A")
location = data.get("Location", "N/A")

# Skills extraction
skills = data.get("Skills", {})
technical_skills = skills.get("Technical Skills", [])
non_technical_skills = skills.get("Non-Technical Skills", [])
technical_skills_str = ", ".join(technical_skills) if technical_skills else "N/A"
non_technical_skills_str = ", ".join(non_technical_skills) if non_technical_skills else "N/A"

# Education extraction
education_list = data.get("Education", [])
education_str = "\n".join([
    f"{edu.get('Degree', 'N/A')} from {edu.get('Institution', 'N/A')} (Graduated: {edu.get('Years', 'N/A')})"
    for edu in education_list
]) if education_list else "N/A"

# Work Experience extraction
work_experience_list = data.get("Work Experience", [])
work_experience_str = "\n".join([
    f"{job.get('Job Title', 'N/A')} at {job.get('Company Name', 'N/A')} ({job.get('Years of Experience', 'N/A')})\nResponsibilities: {', '.join(job.get('Responsibilities', []) if job.get('Responsibilities') else ['N/A'])}"
    for job in work_experience_list
]) if work_experience_list else "N/A"

# Certifications extraction
certifications_str = ", ".join(data.get("Certifications", [])) if data.get("Certifications") else "N/A"

# Languages extraction
languages_str = ", ".join(data.get("Languages spoken", [])) if data.get("Languages spoken") else "N/A"

# Print extracted details
print(f"Full Name: {full_name}")
print(f"Contact Number: {contact_number}")
print(f"Email Address: {email_address}")
print(f"Location: {location}")
print(f"Technical Skills: {technical_skills_str}")
print(f"Non-Technical Skills: {non_technical_skills_str}")
print(f"Education: {education_str}")
print(f"Work Experience: {work_experience_str}")
print(f"Certifications: {certifications_str}")
print(f"Languages: {languages_str}")

Full Name: Preeti
Contact Number: +91-7027425855
Email Address: Jamdagnipreeti24@gmail.com
Location: SECTOR -70, GURUGRAM, HARYANA (122001)
Technical Skills: MS-Word, MS-Excel, Power Point, MS Outlook, Savior Attendance Software, Time Office
Non-Technical Skills: Training & Development, Competency Matrix Review, Training Need  Identification (TNI), Training Calendar Preparation, Training Record Preparation, Training  Feedback  Analysis, Internal  Training Certificate  Preparation, Skill Development, Skill gap  Analysis, On  Job Training  Plan, Skill  matrix  Upgradation, Auditing (Internal & IATF/ISO/EHS/VSA), Recruitment, Talent  Acquisitions, Candidate sourcing, Candidate screening, Interviewing, Negotiation, Onboarding, Employee  satisfaction  survey, Exit Interviews, Exit Formalities, Exit Analysis, Retention strategies, Compliance Management, Recruitments Audits, MIS Updating, KRA  Updating, R&R  and J.D Preparation, Organization  Chart  preparation, Employee  Engagement  Activiti

# Json Format

In [18]:
import json
import re

In [16]:
resume_details = {
    "Full Name": full_name,
    "Contact Number": contact_number,
    "Email Address": email_address,
    "Location": location,
    "Technical Skills": technical_skills_str,
    "Non-Technical Skills": non_technical_skills_str,
    "Education": education_str,
    "Work Experience": work_experience_str,
    "Certifications": certifications_str,
    "Languages": languages_str
}

In [19]:
name = re.sub(r'[^a-zA-Z0-9]', '_', full_name)  
json_filename = f"{name}.json" 

In [20]:
with open(json_filename, "w") as json_file:
    json.dump(resume_details, json_file, indent=4)

print(f"Resume details successfully saved to {json_filename}")

Resume details successfully saved to Preeti.json
